# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [4]:
df = pd.DataFrame(data.groupby(['CustomerID', 'ProductName']).Quantity.sum())
df = df.reset_index()

In [5]:
df

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1
...,...,...,...
63623,98200,Vol Au Vents,50
63624,98200,Wasabi Powder,25
63625,98200,Wine - Fume Blanc Fetzer,25
63626,98200,Wine - Hardys Bankside Shiraz,25


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [6]:
product_by_customer = df.pivot_table(index=["CustomerID"], columns=["ProductName"], values=["Quantity"]).fillna(0)
product_by_customer

Quantity                                    \
ProductName Anchovy Paste - 56 G Tube Appetizer - Mini Egg Roll, Shrimp   
CustomerID                                                                
33                                0.0                               0.0   
200                               0.0                               0.0   
264                               0.0                               0.0   
356                               0.0                               0.0   
412                               0.0                               0.0   
...                               ...                               ...   
97928                             0.0                               0.0   
98069                             0.0                               0.0   
98159                             0.0                               0.0   
98185                             0.0                               0.0   
98200                             0.0                               0.0   

                                                                 \
ProductName Appetizer - Mushroom Tart Appetizer - Sausage Rolls   
CustomerID                                                        
33                                0.0                       0.0   
200                               0.0                       0.0   
264                               0.0                       0.0   
356                               0.0                       0.0   
412                               0.0                       0.0   
...                               ...                       ...   
97928                             0.0                      25.0   
98069                             0.0                      25.0   
98159                             0.0                       0.0   
98185                            25.0                      25.0   
98200                             0.0                       0.0   

                                                               \
ProductName Apricots - Dried Apricots - Halves Apricots Fresh   
CustomerID                                                      
33                       1.0               0.0            0.0   
200                      0.0               0.0            0.0   
264                      0.0               1.0            1.0   
356                      0.0               0.0            0.0   
412                      1.0               0.0            0.0   
...                      ...               ...            ...   
97928                    0.0              50.0            0.0   
98069                    0.0              25.0            0.0   
98159                    0.0               0.0            0.0   
98185                    0.0              25.0            0.0   
98200                    0.0              25.0           25.0   

                                                                          ...  \
ProductName Arizona - Green Tea Artichokes - Jerusalem Assorted Desserts  ...   
CustomerID                                                                ...   
33                          0.0                    0.0               1.0  ...   
200                         0.0                    0.0               0.0  ...   
264                         0.0                    0.0               0.0  ...   
356                         0.0                    0.0               0.0  ...   
412                         0.0                    0.0               0.0  ...   
...                         ...                    ...               ...  ...   
97928                      25.0                    0.0               0.0  ...   
98069                       0.0                    0.0              25.0  ...   
98159                       0.0                    0.0               0.0  ...   
98185                       0.0                    0.0               0.0  ...   
98200                       0.0                    0.0              25.0  .

## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [7]:
#since it's euclidean it's the distance, to get the similarity would need to normalize the data first

def distancia(summary, metric='euclidean'):
    pairwise = pd.DataFrame(
        squareform(pdist(summary, metric)),
        columns = summary.index,
        index = summary.index
    )
    return pairwise

In [8]:
euclidean_dist = distancia(product_by_customer)
euclidean_dist

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0.000000,11.916375,10.488088,11.224972,11.401754,11.090537,12.409674,11.045361,11.269428,11.489125,...,206.871941,213.180675,225.656819,198.232187,230.913404,220.501701,217.188858,228.628520,239.000000,229.773802
200,11.916375,0.000000,11.747340,12.083046,12.569805,12.288206,12.165525,12.083046,11.874342,12.000000,...,206.310446,212.635839,224.697575,197.139544,230.952376,220.202180,215.728997,228.010965,239.037654,229.704158
264,10.488088,11.747340,0.000000,11.489125,11.224972,11.445523,12.000000,11.401754,11.180340,11.747340,...,206.387984,212.946003,225.435135,197.600607,230.371439,219.136943,216.612557,228.081126,238.266657,229.773802
356,11.224972,12.083046,11.489125,0.000000,12.083046,11.789826,12.328828,11.135529,11.958261,12.165525,...,206.649462,213.082144,225.452878,197.494304,231.038958,219.952268,217.437347,228.098663,238.493186,229.464594
412,11.401754,12.569805,11.224972,12.083046,0.000000,11.704700,12.328828,11.135529,11.789826,11.747340,...,206.900942,211.679002,225.572605,197.630969,230.614397,219.733930,217.446545,227.997807,238.396728,228.927936
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,220.501701,220.202180,219.136943,219.952268,219.733930,219.599636,219.538152,219.924987,219.827205,220.070443,...,283.945417,283.945417,302.076149,272.717803,278.388218,0.000000,273.861279,291.547595,306.186218,307.205143
98069,217.188858,215.728997,216.612557,217.437347,217.446545,217.425849,216.903204,217.294731,217.080630,216.751009,...,283.945417,283.945417,295.803989,283.945417,285.043856,273.861279,0.000000,287.228132,297.909382,294.745653
98159,228.628520,228.010965,228.081126,228.098663,227.997807,228.197283,228.028507,228.181945,227.868383,228.103047,...,283.945417,279.508497,300.000000,290.473751,300.000000,291.547595,287.228132,0.000000,304.138127,305.163890


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [9]:
similar_customers = euclidean_dist.sort_values(33).iloc[1:6, :][[33]].index.to_list()
#.sort_values(33) sorting by 33 column, which is customer ID, so here indicate by which customer want to filter
#.iloc[1:6, :] showing 2 to 5 rows, don't want to show the first one because it's the customer itself. : show all column
# [[33]] only show column 33, which has to match with id in sort
#index.to_list() only showing indexes and adding to list
similar_customers
#returning the list that containt customer IDs becuase matrix is done from IDs

[3535, 264, 3317, 2503, 3305]

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [10]:
similar_customer_records = df[df.CustomerID.isin(similar_customers)]
similar_customer_records
#from df taking only customers with these IDs

,CustomerID,ProductName,Quantity
131,264,Apricots - Halves,1
132,264,Apricots Fresh,1
133,264,Bacardi Breezer - Tropical,1
134,264,Bagel - Plain,1
135,264,Banana - Leaves,1
...,...,...,...
3003,3535,Tray - 16in Rnd Blk,1
3004,3535,Wanton Wrap,1
3005,3535,Wine - Blue Nun Qualitatswein,1
3006,3535,"Wine - Magnotta, Merlot Sr Vqa",1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [11]:
product_records = pd.DataFrame(similar_customer_records.groupby('ProductName').Quantity.sum().sort_values(ascending = False))
product_records = product_records.reset_index()
product_records

,ProductName,Quantity
0,Butter - Unsalted,3
1,Soup - Campbells Bean Medley,3
2,Towels - Paper / Kraft,3
3,Wine - Ej Gallo Sierra Valley,3
4,"Pepper - Black, Whole",3
...,...,...
213,"Pepper - White, Ground",1
214,"Pepper - Paprika, Hungarian",1
215,Pate - Cognac,1
216,Pastry - Choclate Baked,1


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [12]:
customer_orders = pd.DataFrame(product_by_customer.loc[33]).reset_index()
customer_orders = customer_orders.rename(columns={33: 'Quantity'})
#have orders of the customer id 33 in df format from the matrix

customer_orders = customer_orders.loc[customer_orders['Quantity'] == 0]
#filter where customer quanitity is 0

products_to_recommend = customer_orders.merge(product_records, on='ProductName')
products_to_recommend = products_to_recommend.rename(columns={'Quantity_x': 'Quantity_customer', 'Quantity_y': 'Quantity_all'})
products_to_recommend = list(products_to_recommend.sort_values('Quantity_all', ascending = False).iloc[0:5, 1].values)
products_to_recommend
#showing the values of the top 5 product recommendations for the customer id 33

['Wine - Blue Nun Qualitatswein',
 'Soup - Campbells Bean Medley',
 'Wine - Ej Gallo Sierra Valley',
 'Butter - Unsalted',
 'Yoghurt Tubes']

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [13]:
all_recom = {}
unique_ids = list(data.CustomerID.unique())

for id in unique_ids:
    similar_customers = euclidean_dist.sort_values(id).iloc[1:6, :][[id]].index.to_list()
    similar_customer_records = df[df.CustomerID.isin(similar_customers)]
    product_records = pd.DataFrame(similar_customer_records.groupby('ProductName').Quantity.sum().sort_values(ascending = False))
    product_records = product_records.reset_index()
    customer_orders = pd.DataFrame(product_by_customer.loc[id]).reset_index()
    customer_orders = customer_orders.rename(columns={id: 'Quantity'})
    customer_orders = customer_orders.loc[customer_orders['Quantity'] == 0]
    products_to_recommend = customer_orders.merge(product_records, on='ProductName')
    products_to_recommend = products_to_recommend.rename(columns={'Quantity_x': 'Quantity_customer', 'Quantity_y': 'Quantity_all'})
    products_to_recommend = list(products_to_recommend.sort_values('Quantity_all', ascending = False).iloc[0:5, 1].values)
    all_recom[id] = ({'product1': products_to_recommend[0], 'product2': products_to_recommend[1],
                      'product3': products_to_recommend[2], 'product4': products_to_recommend[3],
                      'product5': products_to_recommend[4]})
print(all_recom)

{61288: {'product1': 'Chicken - Soup Base', 'product2': 'Jagermeister', 'product3': 'Lime Cordial - Roses', 'product4': 'Macaroons - Two Bite Choc', 'product5': 'Water - Mineral, Natural'}, 77352: {'product1': 'Wine - Valpolicella Masi', 'product2': 'Tahini Paste', 'product3': 'Tuna - Salad Premix', 'product4': 'Chicken - Soup Base', 'product5': 'Isomalt'}, 40094: {'product1': 'Bread - Italian Corn Meal Poly', 'product2': 'Puree - Mocha', 'product3': 'Tuna - Salad Premix', 'product4': 'Sherry - Dry', 'product5': 'Beer - Sleemans Cream Ale'}, 23548: {'product1': 'Squid - Tubes / Tenticles 10/20', 'product2': 'Pernod', 'product3': 'Wanton Wrap', 'product4': 'Bread - Raisin Walnut Oval', 'product5': 'Banana Turning'}, 78981: {'product1': 'Vanilla Beans', 'product2': 'Lettuce - Frisee', 'product3': 'Yogurt - Blueberry, 175 Gr', 'product4': 'Cinnamon Buns Sticky', 'product5': 'Pop Shoppe Cream Soda'}, 83106: {'product1': 'Cocoa Butter', 'product2': 'Cheese - Boursin, Garlic / Herbs', 'produ

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [14]:
recommended_products_euc = pd.DataFrame(all_recom)
recommended_products_euc = recommended_products_euc.T.reset_index().rename(columns={'index': 'id'})
recommended_products_euc

,id,product1,product2,product3,product4,product5
0,61288,Chicken - Soup Base,Jagermeister,Lime Cordial - Roses,Macaroons - Two Bite Choc,"Water - Mineral, Natural"
1,77352,Wine - Valpolicella Masi,Tahini Paste,Tuna - Salad Premix,Chicken - Soup Base,Isomalt
2,40094,Bread - Italian Corn Meal Poly,Puree - Mocha,Tuna - Salad Premix,Sherry - Dry,Beer - Sleemans Cream Ale
3,23548,Squid - Tubes / Tenticles 10/20,Pernod,Wanton Wrap,Bread - Raisin Walnut Oval,Banana Turning
4,78981,Vanilla Beans,Lettuce - Frisee,"Yogurt - Blueberry, 175 Gr",Cinnamon Buns Sticky,Pop Shoppe Cream Soda
...,...,...,...,...,...,...
995,49005,"Pasta - Penne, Rigate, Dry",Browning Caramel Glace,Tea - Decaf Lipton,"Garlic - Primerba, Paste","Wine - Red, Harrow Estates, Cab"
996,41286,Veal - Inside,Wasabi Powder,"Thyme - Lemon, Fresh","Placemat - Scallop, White",Wonton Wrappers
997,85878,Olives - Kalamata,Wine - White Cab Sauv.on,Bread - Calabrese Baguette,Veal - Eye Of Round,Wine - Two Oceans Cabernet
998,68506,Beef - Ground Medium,"Veal - Brisket, Provimi,bnls",Scallops - 10/20,"Rosemary - Primerba, Paste","Oregano - Dry, Rubbed"


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [15]:
#since it's euclidean it's the distance, to get the similarity would need to normalize the data first

def distancia(summary, metric='cosine'):
    pairwise = pd.DataFrame(
        squareform(pdist(summary, metric)),
        columns = summary.index,
        index = summary.index
    )
    return pairwise

In [16]:
cosine_dist = distancia(product_by_customer)
cosine_dist

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0.000000,0.886781,0.774648,0.839772,0.843681,0.848243,0.915650,0.884009,0.875835,0.868135,...,0.885703,0.930549,0.895154,0.955144,0.872026,0.946249,0.891093,0.922307,0.888643,0.897020
200,0.886781,0.000000,0.861622,0.868815,0.918555,0.926067,0.795510,0.935250,0.864456,0.846889,...,0.834109,0.875937,0.801003,0.839743,0.885697,0.915985,0.744661,0.861214,0.900539,0.896524
264,0.774648,0.861622,0.000000,0.879829,0.817627,0.903428,0.855401,0.942005,0.862039,0.907695,...,0.828554,0.902768,0.868942,0.880384,0.808039,0.784998,0.823027,0.857563,0.802031,0.897020
356,0.839772,0.868815,0.879829,0.000000,0.901204,0.908448,0.862918,0.848803,0.934606,0.924994,...,0.864555,0.920991,0.875755,0.872427,0.890811,0.885348,0.922566,0.864967,0.835782,0.865764
412,0.843681,0.918555,0.817627,0.901204,0.000000,0.872402,0.843972,0.825672,0.885161,0.841452,...,0.894287,0.768755,0.890907,0.889368,0.846130,0.863288,0.924455,0.856285,0.828344,0.809510
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.946249,0.915985,0.784998,0.885348,0.863288,0.842051,0.850545,0.875503,0.868375,0.899353,...,0.877321,0.854225,0.912473,0.843081,0.755807,0.000000,0.779202,0.839396,0.881952,0.926312
98069,0.891093,0.744661,0.823027,0.922566,0.924455,0.919993,0.871885,0.901903,0.879989,0.847056,...,0.889526,0.865745,0.885994,0.927741,0.802092,0.779202,0.000000,0.824781,0.844531,0.863140
98159,0.922307,0.861214,0.857563,0.864967,0.856285,0.873163,0.867060,0.866702,0.835112,0.866645,...,0.842378,0.795676,0.867460,0.917521,0.847048,0.839396,0.824781,0.000000,0.840744,0.881655


In [17]:
all_recom_cos = {}
unique_ids = list(data.CustomerID.unique())

for id in unique_ids:
    similar_customers = cosine_dist.sort_values(id).iloc[1:6, :][[id]].index.to_list()
    similar_customer_records = df[df.CustomerID.isin(similar_customers)]
    product_records = pd.DataFrame(similar_customer_records.groupby('ProductName').Quantity.sum().sort_values(ascending = False))
    product_records = product_records.reset_index()
    customer_orders = pd.DataFrame(product_by_customer.loc[id]).reset_index()
    customer_orders = customer_orders.rename(columns={id: 'Quantity'})
    customer_orders = customer_orders.loc[customer_orders['Quantity'] == 0]
    products_to_recommend = customer_orders.merge(product_records, on='ProductName')
    products_to_recommend = products_to_recommend.rename(columns={'Quantity_x': 'Quantity_customer', 'Quantity_y': 'Quantity_all'})
    products_to_recommend = list(products_to_recommend.sort_values('Quantity_all', ascending = False).iloc[0:5, 1].values)
    all_recom_cos[id] = ({'product1': products_to_recommend[0], 'product2': products_to_recommend[1],
                      'product3': products_to_recommend[2], 'product4': products_to_recommend[3],
                      'product5': products_to_recommend[4]})
print(all_recom_cos)

{61288: {'product1': 'Jagermeister', 'product2': 'Wine - Two Oceans Cabernet', 'product3': 'Tofu - Firm', 'product4': 'Appetizer - Sausage Rolls', 'product5': 'Dc - Frozen Momji'}, 77352: {'product1': 'Bread - Rye', 'product2': 'Fondant - Icing', 'product3': 'Lettuce - Frisee', 'product4': 'Bread - French Baquette', 'product5': 'Pork - Kidney'}, 40094: {'product1': 'Juice - V8 Splash', 'product2': 'Beef - Inside Round', 'product3': 'Truffle Cups - Brown', 'product4': 'Bread - Roll, Soft White Round', 'product5': 'Cake - Mini Cheesecake'}, 23548: {'product1': 'Bagel - Plain', 'product2': 'Juice - Orange', 'product3': 'Veal - Sweetbread', 'product4': 'Olives - Stuffed', 'product5': 'Cinnamon Buns Sticky'}, 78981: {'product1': 'Wine - Blue Nun Qualitatswein', 'product2': 'Cookies - Assorted', 'product3': 'Cheese - Cottage Cheese', 'product4': 'Baking Powder', 'product5': 'Quiche Assorted'}, 83106: {'product1': 'Bouq All Italian - Primerba', 'product2': 'Onions - Vidalia', 'product3': 'Sca

In [18]:
recommended_products_cos = pd.DataFrame(all_recom_cos)
recommended_products_cos = recommended_products_cos.T.reset_index().rename(columns={'index': 'id'})
recommended_products_cos

,id,product1,product2,product3,product4,product5
0,61288,Jagermeister,Wine - Two Oceans Cabernet,Tofu - Firm,Appetizer - Sausage Rolls,Dc - Frozen Momji
1,77352,Bread - Rye,Fondant - Icing,Lettuce - Frisee,Bread - French Baquette,Pork - Kidney
2,40094,Juice - V8 Splash,Beef - Inside Round,Truffle Cups - Brown,"Bread - Roll, Soft White Round",Cake - Mini Cheesecake
3,23548,Bagel - Plain,Juice - Orange,Veal - Sweetbread,Olives - Stuffed,Cinnamon Buns Sticky
4,78981,Wine - Blue Nun Qualitatswein,Cookies - Assorted,Cheese - Cottage Cheese,Baking Powder,Quiche Assorted
...,...,...,...,...,...,...
995,49005,Squid - Tubes / Tenticles 10/20,"Pepper - White, Ground",Soup Knorr Chili With Beans,Cheese - Bocconcini,Barramundi
996,41286,"Salsify, Organic","Hickory Smoke, Liquid",Dc Hikiage Hira Huba,Oil - Shortening - All - Purpose,Banana - Leaves
997,85878,Bread - Calabrese Baguette,French Pastry - Mini Chocolate,Foam Dinner Plate,Cheese - Parmesan Grated,Wine - Two Oceans Cabernet
998,68506,Flavouring - Orange,Longos - Chicken Wings,Tia Maria,"Cheese - Brie, Triple Creme",Garbag Bags - Black
